In [38]:
import pandas as pd
import sqlite3
import kagglehub

In [39]:
# Download data from kaggle
file_path = kagglehub.dataset_download("spscientist/students-performance-in-exams")
print("Path to dataset files:", file_path)

Path to dataset files: C:\Users\David\.cache\kagglehub\datasets\spscientist\students-performance-in-exams\versions\1


# ETL Pipeline with student exam data

## Extract

In [40]:
# Extract data from csv file, return pandas dataframe
def extract(file_path):
    data = pd.read_csv(file_path)
    print("Data extraction complete.")
    return data

## Transform

In [41]:
# Transform data and return it
def transform(data):
    # Filter students, include only students that completed the preparation course
    transformed_data = data.loc[data["test preparation course"] == "completed"]
    print("Data transformation complete.")
    return transformed_data

## Load

In [42]:
# Load data into sqlite database
def load(data, db_path):
    # Establish db connection
    con = sqlite3.connect(db_path)
    cur = con.cursor()

    # Create table if not existing
    cur.execute("""
        CREATE TABLE IF NOT EXISTS students (
            id INTEGER PRIMARY KEY AUTOINCREMENT,
            gender VARCHAR(10),
            math_score INTEGER,
            reading_score INTEGER,
            writing_score INTEGER
        )
    """)

    # Insert students into table
    for _, stud in data.iterrows():
        cur.execute("INSERT INTO students (gender, math_score, reading_score, writing_score) VALUES (?, ?, ?, ?)", [stud["gender"], stud["math score"], stud["reading score"], stud["writing score"]])
    print("Inserted "+str(len(data))+" student entries.")

    # Close db connection
    cur.close()
    con.commit()
    con.close()

    print("Data load complete.")

## Composition and Execution

In [43]:
# Combine the etl pipeline into one function
def run_etl_pipeline():
    try:
        # Define paths
        csv_path = file_path+"/StudentsPerformance.csv"
        db_path = "students.db"

        data = extract(csv_path)

        transformed_data = transform(data)

        load(transformed_data, db_path)
    except Exception as e:
        print(f"Error occured: {e}")

In [44]:
# Execute the etl pipeline
run_etl_pipeline()

Data extraction complete.
Data transformation complete.
Inserted 358 student entries.
Data load complete.
